In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
import cartopy.crs as ccrs
from matplotlib.colors import ListedColormap

import cartopy.feature as cfeature
import seaborn as sns 
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.inspection import PartialDependenceDisplay
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from rasterio.plot import show



import os
import geopandas as gpd
import rasterio as rio
from rasterio.plot import plotting_extent


In [ ]:
fire_data = pd.read_csv('Bangala_Fire_full.csv')

# Convert time to datetime

fire_data['time'] = pd.to_datetime(fire_data['time'])
# Remove duplicates based on time, x, and y
fire_data = fire_data.drop_duplicates(subset=['time', 'x', 'y'])
fire_data['Is Fire'] = fire_data['Is Fire'].fillna(0)

#fire_data = fire_data.dropna()

fire_data.rename(columns={'Fuel Type': 'Fuel.Type'}, inplace = True)
fire_data.rename(columns={'Is Fire': 'Is.Fire'}, inplace = True)


# Find the minimum time value (ignition time)
ignition_time = fire_data['time'].min()
fire_data['hours_since_ignition'] = (fire_data['time'] - ignition_time).dt.total_seconds() / 3600
unique_timesteps = fire_data['hours_since_ignition'].unique()

fire_data['Is.Fire_lag1'] = 0

fire_data['Is.Fire_lag2'] = 0

#fire_data['Is.Fire'] = 0


fire_data['Is.Fire_future1'] = 0


In [ ]:
# Convert target_timesteps to datetime format
target_timesteps = pd.to_datetime(['2019-12-04 01:00:00', '2019-12-04 13:00:00', '2019-12-06 06:00:00', 
                       '2019-12-06 18:00:00', '2019-12-07 17:00:00',  '2019-12-08 08:00:00', 
                      '2019-12-08 13:00:00' ])

# Filter the DataFrame based on the target_timesteps
fire_data = fire_data[fire_data['time'].isin(target_timesteps)]


unique_timesteps = np.array([  0.,  12.,  53.,  65.,  88., 103., 108.])

# Ensure the 'time' column in fire_data is also in datetime format
fire_data['time'] = pd.to_datetime(fire_data['time'])


In [ ]:
for i in range(0, len(unique_timesteps)):
    #print("current timestep = ", unique_timesteps[i])
    
    #print(i)
    
    if i != 0 and i!= 6:     
        fire_data.loc[fire_data['hours_since_ignition']==unique_timesteps[i], "Is.Fire_lag1"] = fire_data[fire_data['hours_since_ignition']==unique_timesteps[i-1]]["Is.Fire"].values
        #fire_data.loc[fire_data['hours_since_ignition']==unique_timesteps[i], "hours_diff"] = unique_timesteps[i]-unique_timesteps[i-1]
        fire_data.loc[fire_data['hours_since_ignition']==unique_timesteps[i], "Is.Fire_future1"] = fire_data[fire_data['hours_since_ignition']==unique_timesteps[i+1]][ "Is.Fire"].values

    if i == 0:
        fire_data.loc[fire_data['hours_since_ignition']==unique_timesteps[i], "Is.Fire_future1"] = fire_data[fire_data['hours_since_ignition']==unique_timesteps[i+1]]["Is.Fire"].values

    
    if i ==6:
        fire_data.loc[fire_data['hours_since_ignition']==unique_timesteps[i], "Is.Fire_lag1"] = fire_data[fire_data['hours_since_ignition']==unique_timesteps[i-1]]["Is.Fire"].values
    #    fire_data.loc[fire_data['hours_since_ignition']==unique_timesteps[i], "hours_diff"] = unique_timesteps[i]-unique_timesteps[i-1]
        fire_data.loc[fire_data['hours_since_ignition']==unique_timesteps[i], "Is.Fire_future1"] = fire_data[fire_data['hours_since_ignition']==unique_timesteps[i]]["Is.Fire"].values
        
    if i >= 2:
                fire_data.loc[fire_data['hours_since_ignition']==unique_timesteps[i], "Is.Fire_lag2"] = fire_data[fire_data['hours_since_ignition']==unique_timesteps[i-2]]["Is.Fire"].values
        

In [ ]:
full_fire_data = pd.read_csv('Bangala_Fire_full.csv')
full_fire_data['time'] = pd.to_datetime(full_fire_data['time'])

full_fire_data.rename(columns={'Fuel Type': 'Fuel.Type'}, inplace = True)

full_fire_data = full_fire_data.drop_duplicates(subset=['time', 'x', 'y'])

# Find the minimum time value (ignition time)
ignition_time = full_fire_data['time'].min()
# Calculate hours since ignition for each time entry
full_fire_data['hours_since_ignition'] = (full_fire_data['time'] - ignition_time).dt.total_seconds() / 3600
full_fire_data = full_fire_data.dropna()
# Calculate hours since ignition for each time entry

In [ ]:

columns_to_update = ['wspd', 'wind_direction_deg', 'rh', 'sm', 'st']

for i in range(1, len(unique_timesteps)):
    start_time = unique_timesteps[i - 1]
    end_time = unique_timesteps[i]
    
    # Filter data for the current segment
    segment = full_fire_data[(full_fire_data['hours_since_ignition'] > start_time) & (full_fire_data['hours_since_ignition'] <= end_time)]
    
    if not segment.empty:
        # Find the row with the maximum temperature within the segment
        max_temp_row = segment.loc[segment['st'].idxmax()]
        
        # Update the values in fire_data for the current unique timestep
        mask = fire_data['hours_since_ignition'] == unique_timesteps[i]
        for column in columns_to_update:
            fire_data.loc[mask, column] = max_temp_row[column]



In [ ]:
fire_data['TSF'] = fire_data['TSF'].apply(lambda x: 0 if x == '<10' else 1)


In [ ]:
fire_data = fire_data.dropna()

In [ ]:
fire_data['hours_diff_past'] = 0

fire_data['hours_diff_future'] = 0

In [ ]:
for i in range(0, len(unique_timesteps)):
    #print("current timestep = ", unique_timesteps[i])
    
    #print(i)
    if i != 0 and i!= 6:     
        fire_data.loc[fire_data['hours_since_ignition']==unique_timesteps[i], "hours_diff_past"] = unique_timesteps[i]-unique_timesteps[i-1]

    if i ==6:
        fire_data.loc[fire_data['hours_since_ignition']==unique_timesteps[i], "hours_diff_past"] = unique_timesteps[i]-unique_timesteps[i-1]
    

In [ ]:
for i in range(0, len(unique_timesteps)):
    #print("current timestep = ", unique_timesteps[i])
    
    #print(i)
    if i!= 6:     
        fire_data.loc[fire_data['hours_since_ignition']==unique_timesteps[i], "hours_diff_future"] = unique_timesteps[i+1]-unique_timesteps[i]

    if i == 6:
                fire_data.loc[fire_data['hours_since_ignition']==unique_timesteps[i], "hours_diff_future"] = unique_timesteps[i]-unique_timesteps[i]


    

In [ ]:
firedata_desc = fire_data[['Aspect','Slope', 'Fuel.Type', 'wspd', 'wind_direction_deg',
               'rh', 'sm', 'st', 'TSF']].describe()

print(firedata_desc.to_latex())

In [ ]:
# Get frequency counts and percentages for TSF
tsf_summary = fire_data['TSF'].value_counts().reset_index()
tsf_summary.columns = ['TSF', 'Frequency']
tsf_summary['Percentage'] = 100 * tsf_summary['Frequency'] / len(fire_data)

# Get frequency counts and percentages for Fuel.Type
fuel_summary = fire_data['Fuel.Type'].value_counts().reset_index()
fuel_summary.columns = ['Fuel.Type', 'Frequency']
fuel_summary['Percentage'] = 100 * fuel_summary['Frequency'] / len(fire_data)

# Print to display summary statistics in a readable format
print(tsf_summary)
print(fuel_summary)

In [ ]:
print(tsf_summary.to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.1f}".format,
))

In [ ]:
print(fuel_summary.to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.1f}".format,
))

In [ ]:
#Random Forest VIP and Marginal Effects


X = fire_data[['x', 'y', 'Aspect', 'Slope', 'Fuel.Type', 'wspd', 'wind_direction_deg',
               'rh', 'sm', 'st',  'Is.Fire_lag1', 'Is.Fire_future1', 'hours_since_ignition', 
               'hours_diff_past', 'TSF', 'hours_diff_future']]

y = fire_data['Is.Fire']  # Target variable


# Identify the unique timesteps and find the last one
unique_timesteps = fire_data['hours_since_ignition'].unique()
# Select the second last timestep for testing
test_timestep = unique_timesteps[5]

# Split data into training (all but the last timestep) and testing (second last timestep)
train_data = fire_data
test_data = fire_data[fire_data['hours_since_ignition'] == test_timestep]

# Prepare training and testing sets
X_train = train_data[['x', 'y', 'Aspect', 'Slope', 'Fuel.Type',  'wspd', 'wind_direction_deg',
                      'rh', 'sm', 'st', 'Is.Fire_lag1', 'Is.Fire_future1', 'hours_since_ignition', 
                      'hours_diff_past', 'TSF','hours_diff_future']]

y_train = train_data['Is.Fire']

X_test = test_data[['x', 'y', 'Aspect', 'Slope', 'Fuel.Type', 'wspd', 'wind_direction_deg', 
                    'rh', 'sm', 'st', 'Is.Fire_lag1', 'Is.Fire_future1', 'hours_since_ignition', 
                    'hours_diff_past', 'TSF', 'hours_diff_future']]
y_test = test_data['Is.Fire']

In [ ]:

# Initialize the RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=500)

# Train the model
rf_model.fit(X_train, y_train)

In [ ]:
features = ['hours_since_ignition', 'wspd', 'wind_direction_deg', 'rh', 'sm', 'st','Aspect', 'Slope', 'Fuel.Type']


feature_names = ['Hours Since Ignition', 'Wind Speed', 'Wind Direction', 'Relative Humidiy', 
            'Soil Moisture', 'Surface Temperature','Aspect', 'Slope', "Fuel Type"]

In [ ]:
# Create the plot
fig, ax = plt.subplots(figsize=(20, 15))

# Plot the partial dependence for each feature
disp = PartialDependenceDisplay.from_estimator(rf_model, X_train, features, ax=ax)
plt.suptitle('Marginal Effects Plot', fontsize=22)

# Set feature names as titles and adjust label sizes
for i, ax_ in enumerate(disp.axes_.ravel()):
    ax_.set_xlabel(feature_names[i], fontsize=16)
    ax_.set_ylabel(ax_.get_ylabel(), fontsize=16)

# Overall title and plot adjustments
plt.suptitle('Marginal Effects Plot for Random Forest Fire 3', fontsize=22)
plt.tight_layout(rect=[0, 0, 1, 0.95], w_pad=2)  # Adjust layout with more vertical space
plt.show()

In [ ]:
fire_data['Fuel.Type'].unique() #Interesting to not a spike in fuel type at 150, 100 is forest and 150 
                                # is apparently wet forest, explaining the dip. Quite insightful

In [ ]:
# Create a DataFrame for better visualization
importances = rf_model.feature_importances_
feature_names = ['x', 'y', 'Aspect', 'Slope', 'Fuel Type', 'Wind Speed', 'Wind Direction',
       'Relative Humidity', 'Soil Moisture', 'Surface Temperature', ' Fire Lag', 'Fire Future',
       'Hours Since Ignition', 'Hours Since Last Obs', 'TSF', 'Hours Before Next Obs']
feature_importances = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

# Plot the feature importances
plt.figure(figsize=(10, 6))
plt.barh(feature_importances['Feature'], feature_importances['Importance'], color='skyblue')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importances Plot for Random Forest Fire 3')
plt.gca().invert_yaxis()  # Invert y-axis to show the most important feature at the top
plt.show()

In [ ]:

# Initialize the RandomForestClassifier
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train, y_train)

In [ ]:
# Create the plot
fig, ax = plt.subplots(figsize=(20, 15))
features = ['hours_since_ignition', 'wspd', 'wind_direction_deg', 'rh', 'sm', 'st','Aspect', 'Slope', 'Fuel.Type']


feature_names = ['Hours Since Ignition', 'Wind Speed', 'Wind Direction', 'Relative Humidiy', 
            'Soil Moisture', 'Surface Temperature','Aspect', 'Slope', "Fuel Type"]
# Plot the partial dependence for each feature
disp = PartialDependenceDisplay.from_estimator(dt_model, X_train, features, ax=ax)
plt.suptitle('Decision Tree Marginal Effects Plot Fire 1', fontsize=22)

# Set feature names as titles and adjust label sizes
for i, ax_ in enumerate(disp.axes_.ravel()):
    ax_.set_xlabel(feature_names[i], fontsize=16)
    ax_.set_ylabel(ax_.get_ylabel(), fontsize=16)

# Overall title and plot adjustments
plt.suptitle('Decision Tree Marginal Effects Plot Fire 3', fontsize=22)
plt.tight_layout(rect=[0, 0, 1, 0.95], w_pad=2)  # Adjust layout with more vertical space
plt.show()

In [ ]:
importances = dt_model.feature_importances_
feature_names = ['x', 'y', 'Aspect', 'Slope', 'Fuel Type', 'Wind Speed', 'Wind Direction',
       'Relative Humidity', 'Soil Moisture', 'Surface Temperature', ' Fire Lag', 'Fire Future',
       'Hours Since Ignition', 'Hours Since Last Obs', 'TSF', 'Hours Before Next Obs']
# Create a DataFrame for better sorting and display
feature_importance = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
})

# Sort by importance
feature_importance = feature_importance.sort_values(by='Importance', ascending=True)

# Create the plot
plt.figure(figsize=(10, 6))
plt.barh(feature_importance['Feature'], feature_importance['Importance'], color='lightcoral')
plt.xlabel("Importance")
plt.title("Feature Importance Plot for Decision Tree Fire 3")
#plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

In [ ]:
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)

In [ ]:
coefficients = lr_model.coef_[0]
intercept = lr_model.intercept_[0]

# Create a DataFrame for better visualization
feature_names = X_train.columns if isinstance(X_train, pd.DataFrame) else [f"Feature {i+1}" for i in range(X_train.shape[1])]
summary_table = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients
})

# Adding the intercept separately
intercept_row = pd.DataFrame({
    'Feature': ['Intercept'],
    'Coefficient': [intercept]
})

# Combine and display the table
summary_table = pd.concat([intercept_row, summary_table], ignore_index=True)
print(summary_table.to_latex())

In [ ]:
# Create the plot
fig, ax = plt.subplots(figsize=(20, 15))
features = ['hours_since_ignition', 'wspd', 'wind_direction_deg', 'rh', 'sm', 'st','Aspect', 'Slope', 'Fuel.Type']


feature_names = ['Hours Since Ignition', 'Wind Speed', 'Wind Direction', 'Relative Humidiy', 
            'Soil Moisture', 'Surface Temperature','Aspect', 'Slope', "Fuel Type"]
# Plot the partial dependence for each feature
disp = PartialDependenceDisplay.from_estimator(lr_model, X_train, features, ax=ax)
plt.suptitle('Logistic Regression Marginal Effects Plot Fire 3', fontsize=22)

# Set feature names as titles and adjust label sizes
for i, ax_ in enumerate(disp.axes_.ravel()):
    ax_.set_xlabel(feature_names[i], fontsize=16)
    ax_.set_ylabel(ax_.get_ylabel(), fontsize=16)

# Overall title and plot adjustments
plt.suptitle('Logistic Regression Marginal Effects Plot Fire 3', fontsize=22)
plt.tight_layout(rect=[0, 0, 1, 0.95], w_pad=2)  # Adjust layout with more vertical space
plt.show()

In [ ]:
coefficients = lr_model.coef_[0]

coefficients

In [ ]:
X = fire_data[['x', 'y', 'Aspect', 'Slope', 'Fuel.Type', 'wspd', 'wind_direction_deg',
               'rh', 'sm', 'st',  'Is.Fire_lag1', 'Is.Fire_future1', 'hours_since_ignition', 
               'hours_diff_past', 'TSF', 'hours_diff_future']]
feature_names = ['x', 'y', 'Aspect', 'Slope', 'Fuel Type', 'Wind Speed', 'Wind Direction',
       'Relative Humidity', 'Soil Moisture', 'Surface Temperature', ' Fire Lag', 'Fire Future',
       'Hours Since Ignition', 'Hours Since Last Obs', 'TSF', 'Hours Before Next Obs']

feature_names

In [ ]:

from sklearn.inspection import permutation_importance

# Assuming `lr_model` is trained on unscaled data
result = permutation_importance(lr_model, X_test, y_test, n_repeats=30, random_state=0)

# Extract feature names and importances
importances = result.importances_mean

# Create a DataFrame to organize the results
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=True)

# Plotting
plt.figure(figsize=(10, 6))
plt.barh(importance_df['Feature'], importance_df['Importance'], color='lightgreen')
plt.xlabel("Mean Decrease in Accuracy")
plt.title("Permutation Feature Importance for Logistic Regression Fire 3")
#plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)

In [ ]:
from sklearn.inspection import permutation_importance

result = permutation_importance(knn_model, X_test, y_test, n_repeats=30, random_state=0)

# Extract feature names and importances
feature_names = ['x', 'y', 'Aspect', 'Slope', 'Fuel Type', 'Wind Speed', 'Wind Direction',
       'Relative Humidity', 'Soil Moisture', 'Surface Temperature', ' Fire Lag', 'Fire Future',
       'Hours Since Ignition', 'Hours Since Last Obs', 'TSF', 'Hours Before Next Obs']
importances = result.importances_mean

# Create a DataFrame to organize the results
feature_importance = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
})

# Sort by importance
feature_importance = feature_importance.sort_values(by='Importance', ascending=True)

# Plotting
plt.figure(figsize=(10, 6))
plt.barh(feature_importance['Feature'], feature_importance['Importance'], color='lightblue')
plt.xlabel("Mean Decrease in Accuracy")
plt.title("Permutation Feature Importance for KNN Fire 3")
#plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

In [ ]:
results = {"time": unique_timesteps}

# Initialize DataFrame and columns for each model's metrics
results = pd.DataFrame(data=results)
results['KNN_Accuracy'] = 0
results['KNN_Recall'] = 0
results['KNN_Precision'] = 0

results['RF_Accuracy'] = 0
results['RF_Recall'] = 0
results['RF_Precision'] = 0

results['DT_Accuracy'] = 0
results['DT_Recall'] = 0
results['DT_Precision'] = 0



results['LR_Accuracy'] = 0
results['LR_Recall'] = 0
results['LR_Precision'] = 0



In [ ]:
for i in range(1, len(unique_timesteps)):
    print(i)
    # Select the second last timestep for testing
    test_timestep = unique_timesteps[i]

    # Split data into training (all but the last timestep) and testing (second last timestep)
    train_data = fire_data[fire_data['hours_since_ignition'] != test_timestep]
    test_data = fire_data[fire_data['hours_since_ignition'] == test_timestep]

    # Prepare training and testing sets
    X_train = train_data[['x', 'y', 'Aspect', 'Slope', 'Fuel.Type', 'wspd', 'wind_direction_deg',
                          'rh', 'sm', 'st', 'TSF', 'Is.Fire_lag1', 'Is.Fire_future1', 'hours_since_ignition', 
                          'hours_diff_past', 'hours_diff_future']]
    y_train = train_data['Is.Fire']

    X_test = test_data[['x', 'y', 'Aspect', 'Slope', 'Fuel.Type', 'wspd', 'wind_direction_deg', 
                        'rh', 'sm', 'st', 'TSF', 'Is.Fire_lag1', 'Is.Fire_future1', 'hours_since_ignition', 
                        'hours_diff_past', 'hours_diff_future']]
    y_test = test_data['Is.Fire']
    
    # KNN
    print("KNN")
    knn_model = KNeighborsClassifier(n_neighbors=5)
    knn_model.fit(X_train, y_train)
    y_pred_knn = knn_model.predict(X_test)
    results.loc[results['time'] == test_timestep, 'KNN_Accuracy'] = accuracy_score(y_test, y_pred_knn)
    results.loc[results['time'] == test_timestep, 'KNN_Recall'] = recall_score(y_test, y_pred_knn)
    results.loc[results['time'] == test_timestep, 'KNN_Precision'] = precision_score(y_test, y_pred_knn)

    # Random Forest
    print('RF')
    rf_model = RandomForestClassifier(n_estimators=100)
    rf_model.fit(X_train, y_train)
    y_pred_rf = rf_model.predict(X_test)
    results.loc[results['time'] == test_timestep, 'RF_Accuracy'] = accuracy_score(y_test, y_pred_rf)
    results.loc[results['time'] == test_timestep, 'RF_Recall'] = recall_score(y_test, y_pred_rf)
    results.loc[results['time'] == test_timestep, 'RF_Precision'] = precision_score(y_test, y_pred_rf)

    # Decision Tree
    print('DT')
    dt_model = DecisionTreeClassifier()
    dt_model.fit(X_train, y_train)
    y_pred_dt = dt_model.predict(X_test)
    results.loc[results['time'] == test_timestep, 'DT_Accuracy'] = accuracy_score(y_test, y_pred_dt)
    results.loc[results['time'] == test_timestep, 'DT_Recall'] = recall_score(y_test, y_pred_dt)
    results.loc[results['time'] == test_timestep, 'DT_Precision'] = precision_score(y_test, y_pred_dt)

    '''
    # SVM
    print("SVM")
    svm_model = SVC(kernel='linear', C=1.0)
    svm_model.fit(X_train, y_train)
    y_pred_svm = svm_model.predict(X_test)
    results.loc[results['time'] == test_timestep, 'SVM_Accuracy'] = accuracy_score(y_test, y_pred_svm)
    results.loc[results['time'] == test_timestep, 'SVM_Recall'] = recall_score(y_test, y_pred_svm)
    results.loc[results['time'] == test_timestep, 'SVM_Precision'] = precision_score(y_test, y_pred_svm)
    '''
    

    # Logistic Regression
    print("LR")
    lr_model = LogisticRegression()
    lr_model.fit(X_train, y_train)
    y_pred_lr = lr_model.predict(X_test)
    results.loc[results['time'] == test_timestep, 'LR_Accuracy'] = accuracy_score(y_test, y_pred_lr)
    results.loc[results['time'] == test_timestep, 'LR_Recall'] = recall_score(y_test, y_pred_lr)
    results.loc[results['time'] == test_timestep, 'LR_Precision'] = precision_score(y_test, y_pred_lr)

In [ ]:
results

In [ ]:
avg_accuracy_knn = np.average(results['KNN_Accuracy'])
avg_accuracy_rf = np.average(results['RF_Accuracy'])

avg_recall_knn = np.average(results['KNN_Recall'])
avg_recall_rf = np.average(results['RF_Recall'])



print("KNN Accuracy = ", avg_accuracy_knn, " KNN Recall = ", avg_recall_knn)
print("RF Accuracy = ", avg_accuracy_rf, " RF Recall = ", avg_recall_rf)

In [ ]:

# Calculate the average of each column
avg_results = results.mean()

# Prepare data for plotting
metrics = ['Accuracy', 'Recall', 'Precision']
models = ['KNN', 'RF', 'DT',  'LR']

# Create a DataFrame for plotting
plot_data = {
    'Model': [],
    'Metric': [],
    'Average': []
}

for model in models:
    for metric in metrics:
        plot_data['Model'].append(model)
        plot_data['Metric'].append(metric)
        plot_data['Average'].append(avg_results[f'{model}_{metric}'])

plot_df = pd.DataFrame(plot_data)

# Plotting
plt.figure(figsize=(10, 6))
sns.barplot(x='Model', y='Average', hue='Metric', data=plot_df)

plt.title('Average Accuracy, Recall, and Precision for Each Model', fontsize = 20)
plt.ylabel('Average Value', fontsize = 20)
plt.xlabel('Model', fontsize=20)
plt.xticks(size=20)
plt.yticks(size=20)
plt.ylim(0, 1)
plt.legend(loc='upper right', fontsize=13)
plt.show()


In [ ]:

# Define the range of k values
k_values = range(1, 11,2)

# Initialize DataFrame to store results for each k value
results_knn = pd.DataFrame({'k': k_values})
results_knn['Avg_Accuracy'] = 0
results_knn['Avg_Recall'] = 0
results_knn['Avg_Precision'] = 0
results_knn['Min_Accuracy'] = 0
results_knn['Min_Recall'] = 0
results_knn['Min_Precision'] = 0

# Loop through each k value
for k in k_values:
    accuracy_list = []
    recall_list = []
    precision_list = []
    
    for i in range(2, len(unique_timesteps)):
        # Select the second last timestep for testing
        test_timestep = unique_timesteps[i]

        # Split data into training (all but the last timestep) and testing (second last timestep)
        train_data = fire_data[fire_data['hours_since_ignition'] != test_timestep]
        test_data = fire_data[fire_data['hours_since_ignition'] == test_timestep]

        # Prepare training and testing sets
        X_train = train_data[['x', 'y', 'Aspect', 'Slope', 'Fuel.Type', 'wspd', 'wind_direction_deg',
                              'rh', 'sm', 'st', 'TSF', 'Is.Fire_lag1', 'Is.Fire_future1', 'hours_since_ignition', 
                              'hours_diff_past', 'hours_diff_future']]
        y_train = train_data['Is.Fire']

        X_test = test_data[['x', 'y', 'Aspect', 'Slope', 'Fuel.Type', 'wspd', 'wind_direction_deg', 
                            'rh', 'sm', 'st', 'TSF', 'Is.Fire_lag1', 'Is.Fire_future1', 'hours_since_ignition', 
                            'hours_diff_past', 'hours_diff_future']]
        y_test = test_data['Is.Fire']
        
        # KNN Model
        knn_model = KNeighborsClassifier(n_neighbors=k)
        knn_model.fit(X_train, y_train)
        y_pred_knn = knn_model.predict(X_test)
        
        # Append metrics for this timestep to the lists
        accuracy_list.append(accuracy_score(y_test, y_pred_knn))
        recall_list.append(recall_score(y_test, y_pred_knn))
        precision_list.append(precision_score(y_test, y_pred_knn))
    
    # Calculate average and minimum metrics for this k value and store in DataFrame
    results_knn.loc[results_knn['k'] == k, 'Avg_Accuracy'] = sum(accuracy_list) / len(accuracy_list)
    results_knn.loc[results_knn['k'] == k, 'Avg_Recall'] = sum(recall_list) / len(recall_list)
    results_knn.loc[results_knn['k'] == k, 'Avg_Precision'] = sum(precision_list) / len(precision_list)

    results_knn.loc[results_knn['k'] == k, 'Min_Accuracy'] = min(accuracy_list)
    results_knn.loc[results_knn['k'] == k, 'Min_Recall'] = min(recall_list)
    results_knn.loc[results_knn['k'] == k, 'Min_Precision'] = min(precision_list)
    
    

    

In [ ]:
# Plotting the average metrics
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
sns.lineplot(x='k', y='Avg_Accuracy', data=results_knn, marker='o', label='Avg Accuracy')
sns.lineplot(x='k', y='Avg_Recall', data=results_knn, marker='o', label='Avg Recall')
sns.lineplot(x='k', y='Avg_Precision', data=results_knn, marker='o', label='Avg Precision')

plt.title('Average KNN Performance for Different Values of k')
plt.xlabel('k')
plt.ylabel('Average Metric Value')
plt.legend()
plt.ylim(0.7, 1)
# Plotting the minimum metrics
plt.subplot(1,2,2)
sns.lineplot(x='k', y='Min_Accuracy', data=results_knn, marker='o', label='Min Accuracy')
sns.lineplot(x='k', y='Min_Recall', data=results_knn, marker='o', label='Min Recall')
sns.lineplot(x='k', y='Min_Precision', data=results_knn, marker='o', label='Min Precision')

plt.title('Minimum KNN Performance for Different Values of k')
plt.xlabel('k')
plt.ylabel('Minimum Metric Value')
plt.legend()
plt.ylim(0.7, 1)

plt.tight_layout()
plt.show()

In [ ]:
results_knn

In [ ]:
cmap = ListedColormap(['green', 'orange'])

# Iterate over unique timesteps and create side-by-side plots
for i in range(2, len(unique_timesteps)-1):
    print(f"Iteration: {i}")
    
    # Select the current timestep for testing
    test_timestep = unique_timesteps[i]

    # Split data into training and testing sets
    train_data = fire_data[fire_data['hours_since_ignition'] != test_timestep]
    test_data = fire_data[fire_data['hours_since_ignition'] == test_timestep]

    # Prepare training and testing sets
    X_train = train_data[['x', 'y', 'Aspect', 'Slope', 'Fuel.Type', 'wspd', 'wind_direction_deg',
                          'rh', 'sm', 'st', 'TSF', 'Is.Fire_lag1', 'Is.Fire_future1', 'hours_since_ignition', 
                          'hours_diff_past']]
    y_train = train_data['Is.Fire']

    X_test = test_data[['x', 'y', 'Aspect', 'Slope', 'Fuel.Type', 'wspd', 'wind_direction_deg', 
                        'rh', 'sm', 'st', 'TSF', 'Is.Fire_lag1', 'Is.Fire_future1', 'hours_since_ignition', 
                        'hours_diff_past']]
    y_test = test_data['Is.Fire']
    
    # Train the KNN model and make predictions
    knn_model = KNeighborsClassifier(n_neighbors=5)
    knn_model.fit(X_train, y_train)
    y_pred_test = knn_model.predict(X_test)
    recall = recall_score(y_test, y_pred_test)
    
    # Update the results with the recall
    #results_knn.loc[results_knn['time'] == test_timestep, 'recall'] = recall
    
    # Create side-by-side plots for the true and predicted results
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    # Plot true values
    axes[0].scatter(X_test['x'], X_test['y'], c=y_test, cmap = cmap, s=100, alpha=0.8, edgecolors='w', linewidths=1.5)
    axes[0].set_title('True Is.Fire')
    axes[0].set_xlabel('X')
    axes[0].set_ylabel('Y')
    
    # Plot predicted values
    axes[1].scatter(X_test['x'], X_test['y'], c=y_pred_test, cmap= cmap, s=100, alpha=0.8, edgecolors='w', linewidths=1.5)
    axes[1].set_title('Predicted Is.Fire')
    axes[1].set_xlabel('X')
    axes[1].set_ylabel('Y')
    
    plt.suptitle(f'Timestep: {test_timestep}, recall: {recall:.2f}')
    plt.show()


In [ ]:
cmap = ListedColormap(['green', 'orange'])

# Iterate over unique timesteps and create side-by-side plots
for i in range(1, len(unique_timesteps)-1):
    print(f"Iteration: {i}")
    
    # Select the current timestep for testing
    test_timestep = unique_timesteps[i]

    # Split data into training and testing sets
    train_data = fire_data[fire_data['hours_since_ignition'] != test_timestep]
    test_data = fire_data[fire_data['hours_since_ignition'] == test_timestep]

    # Prepare training and testing sets
    X_train = train_data[['x', 'y', 'Aspect', 'Slope', 'Fuel.Type', 'wspd', 'wind_direction_deg',
                          'rh', 'sm', 'st', 'TSF', 'Is.Fire_lag1', 'Is.Fire_future1', 'hours_since_ignition', 
                          'hours_diff_past']]
    y_train = train_data['Is.Fire']

    X_test = test_data[['x', 'y', 'Aspect', 'Slope', 'Fuel.Type', 'wspd', 'wind_direction_deg', 
                        'rh', 'sm', 'st', 'TSF', 'Is.Fire_lag1', 'Is.Fire_future1', 'hours_since_ignition', 
                        'hours_diff_past']]
    y_test = test_data['Is.Fire']
    
    # Train the KNN model and make predictions
    rf_model = RandomForestClassifier(n_estimators=100)

    # Train the model
    rf_model.fit(X_train, y_train)
    y_pred_test = rf_model.predict(X_test)
    recall = recall_score(y_test, y_pred_test)
    
    # Update the results with the recall
    #results_knn.loc[results_knn['time'] == test_timestep, 'recall'] = recall
    
    # Create side-by-side plots for the true and predicted results
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    # Plot true values
    axes[0].scatter(X_test['x'], X_test['y'], c=y_test, cmap = cmap, s=100, alpha=0.8, edgecolors='w', linewidths=1.5)
    axes[0].set_title('True Is.Fire')
    axes[0].set_xlabel('X')
    axes[0].set_ylabel('Y')
    
    # Plot predicted values
    axes[1].scatter(X_test['x'], X_test['y'], c=y_pred_test, cmap= cmap, s=100, alpha=0.8, edgecolors='w', linewidths=1.5)
    axes[1].set_title('Predicted Is.Fire')
    axes[1].set_xlabel('X')
    axes[1].set_ylabel('Y')
    
    plt.suptitle(f'Timestep: {test_timestep}, recall: {recall:.2f}')
    plt.show()


In [ ]:


# Define colors for TP, FP, TN, FN
colors = {'TP': 'orange', 'FP': 'red', 'TN': 'green', 'FN': 'blue'}

# Create a new column in the test data to classify each point
for i in range(1, len(unique_timesteps) - 1):
    print(f"Iteration: {i}")
    
    # Select the current timestep for testing
    test_timestep = unique_timesteps[i]

    # Split data into training and testing sets
    train_data = fire_data[fire_data['hours_since_ignition'] != test_timestep]
    test_data = fire_data[fire_data['hours_since_ignition'] == test_timestep]

    # Prepare training and testing sets
    X_train = train_data[['x', 'y', 'Aspect', 'Slope', 'Fuel.Type', 'wspd', 'wind_direction_deg',
                          'rh', 'sm', 'st', 'TSF', 'Is.Fire_lag1', 'Is.Fire_future1', 'hours_since_ignition', 
                          'hours_diff_past']]
    y_train = train_data['Is.Fire']

    X_test = test_data[['x', 'y', 'Aspect', 'Slope', 'Fuel.Type', 'wspd', 'wind_direction_deg', 
                        'rh', 'sm', 'st', 'TSF', 'Is.Fire_lag1', 'Is.Fire_future1', 'hours_since_ignition', 
                        'hours_diff_past']]
    y_test = test_data['Is.Fire']
    
    # Train the KNN model and make predictions
    knn_model = KNeighborsClassifier(n_neighbors=5)
    knn_model.fit(X_train, y_train)
    y_pred_test = knn_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred_test)
    
    # Update the results with the accuracy
    #results_knn.loc[results_knn['time'] == test_timestep, 'Accuracy'] = accuracy
    
    # Determine the type of each prediction
    test_data['Prediction_Type'] = np.where((y_test == 1) & (y_pred_test == 1), 'TP', 
                                            np.where((y_test == 0) & (y_pred_test == 0), 'TN',
                                                     np.where((y_test == 0) & (y_pred_test == 1), 'FP', 'FN')))
    
    # Create a color map based on the prediction type
    test_data['Color'] = test_data['Prediction_Type'].map(colors)
    
    # Create scatter plot
    plt.figure(figsize=(8, 6))
    plt.scatter(X_test['x'], X_test['y'], c=test_data['Color'], s=100, alpha=0.8, edgecolors='w', linewidths=1.5)
    
    # Add labels and title
    #plt.xlabel('X', fontsize=20)
    #plt.ylabel('Y', fontsize=20)
    plt.yticks(size=13)
    plt.xticks(size=13)
    plt.title(f'Time Stamp: {test_timestep}', fontsize=36)
    
    # Create a legend
    legend_labels = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=13, label=label)
                     for label, color in colors.items()]
    #plt.legend(handles=legend_labels, title="Prediction Type", fontsize=13)
    
    plt.grid(True)
    plt.savefig(f'KNN_classification_plot_{test_timestep}.png')
    plt.show()


In [ ]:


# Define colors for TP, FP, TN, FN
colors = {'TP': 'orange', 'FP': 'red', 'TN': 'green', 'FN': 'blue'}

# Create a new column in the test data to classify each point
for i in range(1, len(unique_timesteps) - 1):
    print(f"Iteration: {i}")
    
    # Select the current timestep for testing
    test_timestep = unique_timesteps[i]

    # Split data into training and testing sets
    train_data = fire_data[fire_data['hours_since_ignition'] != test_timestep]
    test_data = fire_data[fire_data['hours_since_ignition'] == test_timestep]

    # Prepare training and testing sets
    X_train = train_data[['x', 'y', 'Aspect', 'Slope', 'Fuel.Type', 'wspd', 'wind_direction_deg',
                          'rh', 'sm', 'st', 'TSF', 'Is.Fire_lag1', 'Is.Fire_future1', 'hours_since_ignition', 
                          'hours_diff_past']]
    y_train = train_data['Is.Fire']

    X_test = test_data[['x', 'y', 'Aspect', 'Slope', 'Fuel.Type', 'wspd', 'wind_direction_deg', 
                        'rh', 'sm', 'st', 'TSF', 'Is.Fire_lag1', 'Is.Fire_future1', 'hours_since_ignition', 
                        'hours_diff_past']]
    y_test = test_data['Is.Fire']
    
    # Train the DT model and make predictions
    dt_model = DecisionTreeClassifier()
    dt_model.fit(X_train, y_train)
    y_pred_dt = dt_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred_dt)
    
    # Update the results with the accuracy
    #results_knn.loc[results_knn['time'] == test_timestep, 'Accuracy'] = accuracy
    
    # Determine the type of each prediction
    test_data['Prediction_Type'] = np.where((y_test == 1) & (y_pred_dt == 1), 'TP', 
                                            np.where((y_test == 0) & (y_pred_dt == 0), 'TN',
                                                     np.where((y_test == 0) & (y_pred_dt == 1), 'FP', 'FN')))
    
    # Create a color map based on the prediction type
    test_data['Color'] = test_data['Prediction_Type'].map(colors)
    
    # Create scatter plot
    plt.figure(figsize=(8, 6))
    plt.scatter(X_test['x'], X_test['y'], c=test_data['Color'], s=100, alpha=0.8, edgecolors='w', linewidths=1.5)
    
    # Add labels and title
    #plt.xlabel('X', fontsize=20)
    #plt.ylabel('Y', fontsize=20)
    plt.yticks(size=13)
    plt.xticks(size=13)
    plt.title(f'Time Stamp: {test_timestep}', fontsize=36)
    
    # Create a legend
    legend_labels = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=13, label=label)
                     for label, color in colors.items()]
    #plt.legend(handles=legend_labels, title="Prediction Type", fontsize=13)
    
    plt.grid(True)
    plt.savefig(f'DT_classification_plot_{test_timestep}.png')
    plt.show()


In [ ]:


# Define colors for TP, FP, TN, FN
colors = {'TP': 'orange', 'FP': 'red', 'TN': 'green', 'FN': 'blue'}

# Create a new column in the test data to classify each point
for i in range(1, len(unique_timesteps) - 1):
    print(f"Iteration: {i}")
    
    # Select the current timestep for testing
    test_timestep = unique_timesteps[i]

    # Split data into training and testing sets
    train_data = fire_data[fire_data['hours_since_ignition'] != test_timestep]
    test_data = fire_data[fire_data['hours_since_ignition'] == test_timestep]

    # Prepare training and testing sets
    X_train = train_data[['x', 'y', 'Aspect', 'Slope', 'Fuel.Type', 'wspd', 'wind_direction_deg',
                          'rh', 'sm', 'st', 'TSF', 'Is.Fire_lag1', 'Is.Fire_future1', 'hours_since_ignition', 
                          'hours_diff_past']]
    y_train = train_data['Is.Fire']

    X_test = test_data[['x', 'y', 'Aspect', 'Slope', 'Fuel.Type', 'wspd', 'wind_direction_deg', 
                        'rh', 'sm', 'st', 'TSF', 'Is.Fire_lag1', 'Is.Fire_future1', 'hours_since_ignition', 
                        'hours_diff_past']]
    y_test = test_data['Is.Fire']
    
    # Train the KNN model and make predictions
    lr_model = LogisticRegression()
    lr_model.fit(X_train, y_train)
    y_pred_lr = lr_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred_lr)
    
    # Update the results with the accuracy
    #results_knn.loc[results_knn['time'] == test_timestep, 'Accuracy'] = accuracy
    
    # Determine the type of each prediction
    test_data['Prediction_Type'] = np.where((y_test == 1) & (y_pred_lr == 1), 'TP', 
                                            np.where((y_test == 0) & (y_pred_lr == 0), 'TN',
                                                     np.where((y_test == 0) & (y_pred_lr == 1), 'FP', 'FN')))
    
    # Create a color map based on the prediction type
    test_data['Color'] = test_data['Prediction_Type'].map(colors)
    
    # Create scatter plot
    plt.figure(figsize=(8, 6))
    plt.scatter(X_test['x'], X_test['y'], c=test_data['Color'], s=100, alpha=0.8, edgecolors='w', linewidths=1.5)
    
    # Add labels and title
    #plt.xlabel('X', fontsize=20)
    #plt.ylabel('Y', fontsize=20)
    plt.yticks(size=13)
    plt.xticks(size=13)
    plt.title(f'Time Stamp: {test_timestep}', fontsize=36)
    
    # Create a legend
    legend_labels = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=13, label=label)
                     for label, color in colors.items()]
    #plt.legend(handles=legend_labels, title="Prediction Type", fontsize=13)
    
    plt.grid(True)
    plt.savefig(f'LR_classification_plot_{test_timestep}.png')
    plt.show()


In [ ]:



# Define colors for TP, FP, TN, FN
colors = {'TP': 'orange', 'FP': 'red', 'TN': 'green', 'FN': 'blue'}

# Create a new column in the test data to classify each point
for i in range(1, len(unique_timesteps) - 1):
    print(f"Iteration: {i}")
    
    # Select the current timestep for testing
    test_timestep = unique_timesteps[i]

    # Split data into training and testing sets
    train_data = fire_data[fire_data['hours_since_ignition'] != test_timestep]
    test_data = fire_data[fire_data['hours_since_ignition'] == test_timestep]

    # Prepare training and testing sets
    X_train = train_data[['x', 'y', 'Aspect', 'Slope', 'Fuel.Type', 'wspd', 'wind_direction_deg',
                          'rh', 'sm', 'st', 'TSF', 'Is.Fire_lag1', 'Is.Fire_future1', 'hours_since_ignition', 
                          'hours_diff_past']]
    y_train = train_data['Is.Fire']

    X_test = test_data[['x', 'y', 'Aspect', 'Slope', 'Fuel.Type', 'wspd', 'wind_direction_deg', 
                        'rh', 'sm', 'st', 'TSF', 'Is.Fire_lag1', 'Is.Fire_future1', 'hours_since_ignition', 
                        'hours_diff_past']]
    y_test = test_data['Is.Fire']
    
    # Train the KNN model and make predictions
    rf_model = RandomForestClassifier(n_estimators=500)

    rf_model.fit(X_train, y_train)
    y_pred_test = rf_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred_test)
    
    # Update the results with the accuracy
    #results_knn.loc[results_knn['time'] == test_timestep, 'Accuracy'] = accuracy
    
    # Determine the type of each prediction
    test_data['Prediction_Type'] = np.where((y_test == 1) & (y_pred_test == 1), 'TP', 
                                            np.where((y_test == 0) & (y_pred_test == 0), 'TN',
                                                     np.where((y_test == 0) & (y_pred_test == 1), 'FP', 'FN')))
    
    # Create a color map based on the prediction type
    test_data['Color'] = test_data['Prediction_Type'].map(colors)
    
    # Create scatter plot
    plt.figure(figsize=(8, 6))
    plt.scatter(X_test['x'], X_test['y'], c=test_data['Color'], s=100, alpha=0.8, edgecolors='w', linewidths=1.5)
    
    # Add labels and title
    #plt.xlabel('X', fontsize=20)
    #plt.ylabel('Y', fontsize=20)
    plt.yticks(size=13)
    plt.xticks(size=13)
    plt.title(f'Time Stamp: {test_timestep}', fontsize=36)
    
    # Create a legend
    legend_labels = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=13, label=label)
                     for label, color in colors.items()]
    #plt.legend(handles=legend_labels, title="Prediction Type", fontsize=13)
    
    plt.grid(True)
    plt.savefig(f'RF_classification_plot_{test_timestep}.png')
    plt.show()


In [ ]:
import matplotlib.image as mpimg

image_paths = [
    'LR_classification_plot_12.0.png','LR_classification_plot_53.0.png',
    'LR_classification_plot_65.0.png','LR_classification_plot_88.0.png',
    'LR_classification_plot_103.0.png',
    'RF_classification_plot_12.0.png','RF_classification_plot_53.0.png',
    'RF_classification_plot_65.0.png','RF_classification_plot_88.0.png',
    'RF_classification_plot_103.0.png',
    'KNN_classification_plot_12.0.png','KNN_classification_plot_53.0.png',
    'KNN_classification_plot_65.0.png','KNN_classification_plot_88.0.png',
    'KNN_classification_plot_103.0.png',
    'DT_classification_plot_12.0.png','DT_classification_plot_53.0.png',
    'DT_classification_plot_65.0.png','DT_classification_plot_88.0.png',
    'DT_classification_plot_103.0.png',
    
    
 
]
titles = ["Logistic Regression", "Random Forest", "K Nearest Neighbours", "Decision Trees"]

# Create a figure with 5 rows and 4 columns
fig, axes = plt.subplots(5, 4, figsize=(16, 20))

# Loop through the axes and image paths
for col in range(4):
    for row in range(5):
        idx = row + col * 5
        img = mpimg.imread(image_paths[idx])
        axes[row, col].imshow(img)
        axes[row, col].axis('off')

    # Set the column titles
    axes[0, col].set_title(titles[col], fontsize=26)
    
colors = {'TP': 'orange', 'FP': 'red', 'TN': 'green', 'FN': 'blue'}
legend_labels = [
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=13, label=label)
    for label, color in colors.items()
]
fig.legend(handles=legend_labels, loc='lower center', ncol=4, fontsize=16, frameon=False)
fig.text(0.5, 0.03, 'X', ha='center', fontsize=18)
fig.text(-0.01, 0.5, 'Y', va='center', rotation='vertical', fontsize=18)


# Adjust layout and save the figure
plt.tight_layout(pad=0.02)
plt.savefig("grid_of_images.png", dpi=300)
plt.show()
